# Deep Learning to calculate T$_{GPU}$ and C$_{total}$

In [ ]:
################################################################
# FUNCTIONS FOR MODEL BUILDING, TRAINING, AND EVALUATION
################################################################

################################################################
# Loading necessary libraries
################################################################

import time
import tensorflow as tf
import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.base import BaseEstimator, RegressorMixin, clone
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.neural_network import MLPRegressor
import os
from tensorflow.keras.optimizers import AdamW
import warnings
from sklearn.exceptions import DataConversionWarning
import sys
from tensorflow.keras import layers, Model, Input
from tensorflow.keras import layers, models, regularizers, callbacks, optimizers
from tensorflow.keras.callbacks import EarlyStopping
import json

max_processors = os.cpu_count()

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DataConversionWarning)

GREEN_TEXT = "\033[92m"
BLUE_TEXT = "\033[94m" 
RED_TEXT = "\033[91m"
RESET_TEXT = "\033[0m"

################################################################
# Loading Input Data
################################################################


def get_file_from_user():
    supported_extensions = ['.csv', '.xlsx', '.xls', '.txt']

    while True:
        choice = input("Do you want to load the file from your home directory? (y/n): ").strip().lower()
    
        if choice in ['y', 'yes']:
            directory = os.path.expanduser("~") + "/"
            break
        elif choice in ['n', 'no']:
            directory = input("Enter the full path to the file (use '/' or '\\\\' as needed): ").strip()
            if not directory.endswith(('/', '\\')):
                directory += '/'
            break
        else:
            print("Invalid choice. Please enter 'y' or 'n'.")

    while True:
        file_name = input("Enter the file name *including extension* (e.g., data.xls, data.xlsx, input.csv, test.txt): ").strip()

        if not any(file_name.lower().endswith(ext) for ext in supported_extensions):
            print(f"{RED_TEXT} Invalid file extension. Supported formats: {', '.join(supported_extensions)}{RESET_TEXT}")
            continue

        full_path = os.path.join(directory, file_name)

        if not os.path.exists(full_path):
            print(f"{RED_TEXT} Error: File '{file_name}' does not exist at '{directory}'{RESET_TEXT}")
        else:
            print(f"Loading file from: {full_path}")
            return full_path

def load_data(file_path):
    ext = file_path.lower().split('.')[-1]

    try:
        if ext == 'csv':
            df = pd.read_csv(file_path)
        elif ext in ['xls', 'xlsx']:
            df = pd.read_excel(file_path)
        elif ext == 'txt':
            df = pd.read_csv(file_path, delimiter='\t', engine='python')
        else:
            print(f"{RED_TEXT}Unsupported file format: .{ext}{RESET_TEXT}")
            return None, None, None
    except Exception as e:
        print(f"{RED_TEXT} Failed to load file: {e}{RESET_TEXT}")
        return None, None, None

    # Check for missing values
    if df.isnull().values.any():
        print(f"{RED_TEXT}Error: The file '{file_path}' contains missing values. Exiting the program.{RESET_TEXT}")
        sys.exit(1)

    print(f"{GREEN_TEXT} File loaded successfully. No missing values found.{RESET_TEXT}")

    # Show columns
    print("\n Current columns in the dataset:")
    print(*df.columns.tolist(), sep=", ")

    # Ask if user wants to drop any columns
    drop_choice = input("\nDo you want to drop any columns before selecting inputs/outputs? (y/n): ").strip().lower()
    while drop_choice not in ['y', 'n']:
        print(f"{RED_TEXT}Invalid input. Please enter 'y' or 'n'.{RESET_TEXT}")
        drop_choice = input("Do you want to drop any columns? (y/n): ").strip().lower()

    if drop_choice == 'y':
        while True:
            try:
                num_cols_to_drop = int(input("How many columns do you want to drop (e.g., 1, 2, 3 .... more): ").strip())
                if num_cols_to_drop <= 0:
                    raise ValueError
                break
            except ValueError:
                print(f"{RED_TEXT}Invalid input. Please enter a positive integer.{RESET_TEXT}")

        cols_to_drop = []
        for i in range(num_cols_to_drop):
            while True:
                col_name = input(f"Enter the name of column {i+1} to drop: ").strip()
                if col_name in df.columns:
                    cols_to_drop.append(col_name)
                    break
                else:
                    print(f"{RED_TEXT}Error: Column '{col_name}' not found. Please enter a valid column name.{RESET_TEXT}")

        df.drop(columns=cols_to_drop, inplace=True)
        print(f"{GREEN_TEXT}Successfully dropped columns: {cols_to_drop}{RESET_TEXT}")

    # After dropping, ask user for input and output column names
    print(f"{GREEN_TEXT}\n Updated columns:{RESET_TEXT}")
    print(*df.columns.tolist(), sep=", ")
    
    # Input columns
    while True:
        input_cols = input("Enter input (X) column names separated by commas: ").strip().split(',')
        input_cols = [col.strip() for col in input_cols]
        if all(col in df.columns for col in input_cols):
            break
        else:
            print(f"{RED_TEXT}One or more input columns are invalid. Try again.{RESET_TEXT}")

    # Output column
    while True:
        output_col = input("Enter the output (y) column name: ").strip()
        if output_col in df.columns and output_col not in input_cols:
            break
        else:
            print(f"{RED_TEXT}Invalid or duplicate output column. Try again.{RESET_TEXT}")

    X_data = df[input_cols]
    y_data = df[output_col]
    
    return X_data, y_data, df
    
################################################################
# Creating arrays form the datafile for data preprocessing.
################################################################
data_file = get_file_from_user()

if data_file:
    X, y, NN_df = load_data(data_file)
    if NN_df is not None:
        print(X.head())
Unit = input("Enter the unit for the (Y) column (for no unit leave blank)")
##########################################################################
# Checking if number of parameters are more than datapoint (underfit)
##########################################################################

while True:
    try:
        data_points = X.shape[0]  # Number of data points.
        num_params = X.shape[1]  # Number of parameters.
        
        # Check condition if true, the code will exit with an error.
        if num_params > data_points:
            print(f"{RED_TEXT}Error: Number of parameters ({num_params}) exceeds the number of data points ({data_points}). Please adjust your input datafile. {RESET_TEXT}")
            sys.exit(1)
        
        # Proceed with further processing if datapoints are more than parameters.
        else:
            print(f"{GREEN_TEXT}Data is valid. Proceeding with execution.{RESET_TEXT}")
            break
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)
        
################################################################
# Determine the test size based on data points
################################################################

if data_points < 20:
    TEST_SIZE = 0.15
elif 20 <= data_points < 30:
    TEST_SIZE = 0.2
elif 30 <= data_points < 40:
    TEST_SIZE = 0.25
else:
    TEST_SIZE = 0.3

print(f"{BLUE_TEXT}Number of data points: {data_points}{RESET_TEXT}")
print(f"{BLUE_TEXT}Test size selected: {TEST_SIZE}{RESET_TEXT}")

################################################################
# Random state for data processing
################################################################

RANDOM_STATE=42

################################################################
# File path for saving the models and figures
################################################################
print("\n")
save_location_choice = input("Do you want to save the file in the home directory? (y/n): ").strip().lower()

if save_location_choice in ['y', 'yes']:
    base_dir = os.path.expanduser('~')   # just home directory
elif save_location_choice in ['n', 'no']:
    while True:
        base_dir = input("Enter full directory path where you want to save the file: ").strip()
        if not os.path.exists(base_dir):
            print(f"{RED_TEXT}Directory does not exist! Please provide a valid path.{RESET_TEXT}")
        elif not os.path.isdir(base_dir):
            print(f"{RED_TEXT}Path exists but is not a directory. Try again.{RESET_TEXT}")
        else:
            break
else:
    print("Invalid choice. Exiting.")
    exit(1)

# Create DNN folder inside base_dir
save_dir = os.path.join(base_dir, "DNN")
os.makedirs(save_dir, exist_ok=True)
print(f"{GREEN_TEXT}Directory created or already exists at: {save_dir}{RESET_TEXT}")

# Prompt for valid filename
while True:
    savename = input("Enter the name to save the results in Excel file (without .xlsx extension): ").strip()
    if not savename:
        print(f"{RED_TEXT}Filename cannot be empty. Please enter a valid name.{RESET_TEXT}")
    elif any(char in savename for char in r'\/:*?"<>|'):
        print(f"{RED_TEXT}Invalid characters in filename! Avoid \\ / : * ? \" < > |{RESET_TEXT}")
    else:
        full_path = os.path.join(save_dir, savename + ".xlsx")
        if os.path.exists(full_path):
            overwrite = input(f"{YELLOW_TEXT}File '{savename}.xlsx' already exists in 'DNN'. Overwrite? (y/n): {RESET_TEXT}").strip().lower()
            if overwrite == 'y':
                break
            else:
                print("Please choose a different filename.")
        else:
            break

print(f"{GREEN_TEXT}File will be saved at: {full_path}{RESET_TEXT}")
start_time_total = time.time()
print("\n")
################################################################
# Function to normalize the Input data using MinMaxScaler
# Set range to (0, 1) for easy convergence
################################################################

def scale_data_x(X):
    scaler_X_NN = StandardScaler()
    X_scaled = scaler_X_NN.fit_transform(X)
    return X_scaled, scaler_X_NN

################################################################
# Function to normalize the Output data using MinMaxScaler
# Set range to (0, 1) for easy convergence
################################################################

def scale_data_y(y):
    scaler_y_NN = StandardScaler()
    y_scaled = scaler_y_NN.fit_transform(y.values.reshape(-1, 1))
    return y_scaled, scaler_y_NN


################################################################
# Scaling data
################################################################

X_scaled, scaler_X_NN = scale_data_x(X)
y_scaled, scaler_y_NN = scale_data_y(y)

######################################################################################
# Function for splitting training and testing data based on predetermined test size
######################################################################################

def split_data(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE):
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

################################################################
# Spliting data for training and testing
################################################################

X_train, X_test, y_train, y_test = split_data(X_scaled, y_scaled)


################################################################
# Customized loss function for weighted loss
################################################################

# Root Mean Squared Error (RMSE)
def root_mean_squared_error(y_true, y_pred):
    error = y_pred - y_true
    mse = np.mean(error ** 2)
    return np.sqrt(mse)

# Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    error = y_pred - y_true
    return np.mean(np.abs(error))
    
# R2 score function.
def R2(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    return r2

################################################################
# Funtion to calculate total number of hidden layers
################################################################

def calculate_total_neurons(layers):
    return sum(layers)

################################################################
# User Input for Hyperparameters
################################################################

while True:
    user_choice = input("Would you like to manually enter hyperparameters or use automatic tuning with GridSearchCV? (enter 'manual' or 'auto'): ").strip().lower()
    
    if user_choice == 'manual':
        
        # Displaying important note before collecting inputs.
        print("\033[1mNote: These hyperparameters are defined as per the problem and data complexity.\033[0m\n")
        print("\033[1mChanging these may cause variations in results.\033[0m\n")
        print("\033[1mALL VALUES ARE CASE SENSITIVE.\033[0m\n")

        # Function to validate input as an integer within a specified range.
        def get_integer_input(prompt, min_val, max_val, preferred_val=None):
            while True:
                try:
                    value = input(f"{prompt} (Range: {max_val}-{min_val}, Can try: {preferred_val}): ")
                    value = int(value)
                    if min_val <= value <= max_val:
                        return value
                    else:
                        print(f"{RED_TEXT}Please enter a value between {min_val} and {max_val}.{RESET_TEXT}")
                except ValueError:
                    print(f"{RED_TEXT}Invalid input. Please enter an integer.{RESET_TEXT}")

        # Function to validate input as a float within a specified range.
        def get_float_input(prompt, min_val, max_val, preferred_val=None):
            while True:
                try:
                    value = input(f"{prompt} (Range: {max_val}-{min_val}, Can try: {preferred_val}): ")
                    value = float(value)
                    if min_val <= value <= max_val:
                        return value
                    else:
                        print(f"{RED_TEXT}Please enter a value between {min_val} and {max_val}.{RESET_TEXT}")
                except ValueError:
                    print(f"{RED_TEXT}Invalid input. Please enter a float.{RESET_TEXT}")

        # Function for hidden layers.
        def get_hidden_layers():
            hidden_layers = []
            
            # First mandatory hidden layer.
            first_layer = get_integer_input(
                "Enter number of neurons for the first hidden layer (Can try: 128)", 
                4, 4096, preferred_val=128
            )
            hidden_layers.append(first_layer)
            
            # Optional second and third layers..
            print("Enter number of neurons for the second and third hidden layers in descending order (multiples of 2, max 4096, ending at any multiple of 2 except 2). Each layer is optional.")
            
            preferred_layers = [64, 32]  # Preferred values for the second and third layers.
            for i in range(2):  # Loop for the next two layers.
                while True:
                    cont = input(f"Do you want to add layer {i+2} (Can try: {preferred_layers[i]} if available)? (y/n): ").strip().lower()
                    if cont not in ['y', 'n']:
                        print(f"{RED_TEXT}Invalid input. Please enter 'y' or 'n'.{RESET_TEXT}")
                    else:
                        break
                if cont == 'n':
                    break
                while True:
                    layer = get_integer_input(
                        f"Enter number of neurons for layer {i+2}",
                        4, 4096, preferred_val=preferred_layers[i]
                    )
                    if layer == 2:
                        print(f"{RED_TEXT}Number of neurons in hidden-layer cannot be 2. Please enter a different value.{RESET_TEXT}")
                    elif layer >= hidden_layers[-1]:
                        print(f"{RED_TEXT}Please enter a smaller value than the previous layer ({hidden_layers[-1]}).{RESET_TEXT}")
                    else:
                        hidden_layers.append(layer)
                        break

            # Additional layers beyond the third layer.
            if len(hidden_layers) >= 3:
                while True:
                    while True:
                        cont = input("Do you want to add another layer? (y/n): ").strip().lower()
                        if cont not in ['y', 'n']:
                            print(f"{RED_TEXT}Invalid input. Please enter 'y' or 'n'.{RESET_TEXT}")
                        else:
                            break
                    if cont == 'n':
                        break
                    while True:
                        layer = get_integer_input("Enter number of neurons for the layer: ", 4, 4096)
                        if layer == 2:
                            print(f"{RED_TEXT}Number of neurons in hidden-layer cannot be 2. Please enter a different value.{RESET_TEXT}")
                        elif layer >= hidden_layers[-1]:
                            print(f"{RED_TEXT}Please enter a smaller value than the previous layer ({hidden_layers[-1]}).{RESET_TEXT}")
                        else:
                            hidden_layers.append(layer)
                            break
            return hidden_layers

        # Collecting other parameters from user.
        
        LEARNING_RATE = get_float_input("Enter the learning rate", 0.00001, 0.5, preferred_val=0.01)
        NUM_EPOCHS = get_integer_input("Enter the number of epochs (iterations)", 50, 1000, preferred_val=100)
        BATCH_SIZE = get_integer_input("Enter the batch size", 2, 512, preferred_val=2)
        RANDOM_STATE = get_integer_input("Enter the random state", 0, 314, preferred_val=42)
        
        # Finalize hidden layers based on user input.
        HIDDEN_LAYERS = get_hidden_layers()

        # Display the chosen hyperparameters.
        print(f"{BLUE_TEXT} \n Chosen Hyperparameters: {RESET_TEXT}")
        print(f"{BLUE_TEXT} Hidden Layers With Neurons: {HIDDEN_LAYERS} {RESET_TEXT}")
        print(f"{BLUE_TEXT} Learning Rate: {LEARNING_RATE} {RESET_TEXT}")
        print(f"{BLUE_TEXT} Number of Epochs (iterations): {NUM_EPOCHS} {RESET_TEXT}")
        print(f"{BLUE_TEXT} Batch Size: {BATCH_SIZE} {RESET_TEXT}")
        print(f"{BLUE_TEXT} Test Size: {TEST_SIZE} {RESET_TEXT}")
        print("\n")
        break

    elif user_choice == 'auto':
        
        # Determining the number of processor for grid search.
        while True:
            try:
                num_processors = int(input("Input the number of CPU processors you want to use for parallel processing (For all use -1): "))
                if num_processors == -1 or (num_processors > 0 and num_processors <= max_processors):
                    break
                else:
                    if num_processors > max_processors:
                        print(f"{RED_TEXT}Error: The number exceeds the available processors ({max_processors}).{RESET_TEXT}")
                    else:
                        print(f"{RED_TEXT}Error: Please enter -1 or an integer greater than 0.{RESET_TEXT}")
            except ValueError:
                print(f"{RED_TEXT}Error: Invalid input. Please enter an integer.{RESET_TEXT}")
            
        print(f"{GREEN_TEXT}\nAutomatically selecting NN hyperparameters with GridSearchCV...{RESET_TEXT}")

        #######################
        # GridSearchCV Setup.
        #######################
        class TrackedMLP(BaseEstimator, RegressorMixin):
            def __init__(self, **kwargs):
                self.model = MLPRegressor(**kwargs)
                self.n_iter_ = None
                self.loss_curve_ = None
    
            def fit(self, X, y):
                self.model.fit(X, y)
                self.n_iter_ = self.model.n_iter_
                self.loss_curve_ = self.model.loss_curve_
                return self
    
            def predict(self, X):
                return self.model.predict(X)
    
            def get_params(self, deep=True):
                return self.model.get_params(deep)
    
            def set_params(self, **params):
                self.model.set_params(**params)
                return self
        
        scorings = {
            'MAE': make_scorer(mae, greater_is_better=False),
            'RMSE': make_scorer(root_mean_squared_error, greater_is_better=False),
            'R2': make_scorer(R2, greater_is_better=True),
        }
        
        #######################
        # Parameter Grid Blocks
        #######################
        
        # Learning rates: log-spaced from 1e-1 to 1e-6
        learning_rates = np.logspace(-1, -3, num=3).tolist()

        # Hidden layer architectures grouped by complexity
        hidden_layer_blocks = {
            "shallow": [
                (32,), (64,), (128,), (256,),             # single-layer
                (32, 16), (64, 32), (128, 64),            # 2-layer shallow
                (32, 16, 8), (64, 32, 16), (128, 64, 32)  # 3-layer shallow
            ]

            "medium": [
                (512,), (1024,),
                (64, 32), (128, 64), (256, 128)
            ],
            "deep": [
                (512, 256), (128, 64, 32), (256, 128, 64),
                (512, 256, 128), (256, 128, 64, 32)
            ],
            "very_deep": [
                (512, 256, 128, 64), 
                (512, 256, 128, 64, 32), 
                (512, 256, 128, 64, 32, 16),
                (512, 256, 128, 64, 32, 16, 8)
            ]
        }

        # Batch size groups
        batch_small  = [2, 4, 8, 16]

        # Epochs by depth
        epochs_shallow   = [100, 200, 300, 400, 500]
        epochs_medium    = [600, 800]
        epochs_deep      = [1000, 1500]

        # Final single param_grid with blocks
        param_grid = [
            {   # Shallow nets: high LR, small/medium batch, fewer epochs
            'hidden_layer_sizes': hidden_layer_blocks["shallow"],
            'learning_rate_init': learning_rates,
            'batch_size': batch_small + batch_medium,
            'max_iter': epochs_shallow + epochs_medium + epochs_deep
        },
        {   # Medium nets: moderate LR, medium batch, moderate epochs
            'hidden_layer_sizes': hidden_layer_blocks["medium"],
            'learning_rate_init': learning_rates,
            'batch_size': batch_small + batch_medium,
            'max_iter': epochs_shallow + epochs_medium
        },
        {   # Deep nets: low LR, medium/large batch, longer epochs
            'hidden_layer_sizes': hidden_layer_blocks["deep"],
            'learning_rate_init': learning_rates,
            'batch_size': batch_small + batch_medium,
            'max_iter': epochs_shallow + epochs_medium 
        },
        {   # Very deep nets: tiny LR, large batch, max epochs
            'hidden_layer_sizes': hidden_layer_blocks["very_deep"],
            'learning_rate_init': learning_rates,
            'batch_size': batch_small + batch_medium,
            'max_iter': epochs_shallow 
        }
        ]
        ##############################################
        # Model Setup for MLP regressor.
        ##############################################
        
        model_1 = TrackedMLP(random_state = RANDOM_STATE, early_stopping=False)
        
        ##############################################
        # Performing Grid Search CV for each block.
        ##############################################
        
        grid_search = GridSearchCV(
            estimator=model_1, 
            param_grid=param_grid, 
            scoring=scorings,  # Use custom scoring function
            refit="MAE",
            cv=KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE),
            n_jobs=num_processors, 
            verbose=3,
            return_train_score=True
            )

        # Progress Tracking for GridSearchCV.
        print("Starting Grid Search...")
        
        grid_search.fit(X_scaled, y_scaled.ravel())

        # Extract best model (if early-stopped)
        best_model = grid_search.best_estimator_
        
        # Actual early-stopped iteration count and loss curve from wrapped model
        if hasattr(best_model, "n_iter_"):
            actual_iter = best_model.n_iter_
            print(f"\n Best model stopped after {actual_iter} iterations.")

        if hasattr(best_model, "loss_curve_"):
            final_loss = best_model.loss_curve_[-1] if best_model.loss_curve_ else None
            print(f" Final training loss after early stopping: {final_loss:.6f}")
        
        
        joblib.dump(grid_search, os.path.join(save_file_path, 'grid_search_checkpoint.pkl'))
        
        cv_results = grid_search.cv_results_
        
        ###############################################################
        # Saving results to dataframe and filterring overfitting
        ###############################################################
        # Access the complete results in DataFrame format and Saved results for further analysis.
        # List of metrics to correct in Excel file, as grid search include -ve sign infront of minimization metrices.
        results_df = pd.DataFrame(grid_search.cv_results_)
        results_df['best_n_iter'] = actual_iter
        results_df['best_final_loss'] = final_loss
        
        # Overfitting threshold 
        overfit_threshold = 0.07

        results_df['train_test_gap'] = results_df['mean_train_R2'] - results_df['mean_test_R2']

        # Filter non-overfitted results
        filtered_results_df = results_df[results_df['train_test_gap'] <= overfit_threshold]

        if filtered_results_df.empty:
            print("All models appear overfitted. Using original best model from GridSearchCV.")
            best_model = grid_search.best_estimator_
            best_params_1 = grid_search.best_params_
            best_loss = grid_search.best_score_
        
        else:
            # Pick best from filtered (non-overfit) results
            best_row = filtered_results_df.sort_values(by='mean_test_MAE', ascending=False).iloc[0]
            best_params_1 = best_row['params']
            best_loss = best_row['mean_test_MAE']

        
        # Save the best parameters to specific variables.
        LEARNING_RATE = best_params_1['learning_rate_init']
        NUM_EPOCHS = best_params_1['max_iter']
        BATCH_SIZE = best_params_1['batch_size']
        HIDDEN_LAYERS = best_params_1['hidden_layer_sizes']
        
        print(f"{BLUE_TEXT}\n Best hyperparameters test: {best_params_1}{RESET_TEXT}")
        print(f"{BLUE_TEXT}\n Best MAE loss score test: {np.abs(best_loss)}{RESET_TEXT}")
        
                        
        
        # For Examples (columns containing "MAE", "RMSE").
        error_metrics = [col for col in results_df.columns if any(err in col for err in ["MAE", "RMSE"])]

        # Convert negative error values to positive (absolute values).
        results_df[error_metrics] = results_df[error_metrics].abs()
        results_df.to_excel(os.path.join(save_file_path, f'{savename}_GridSearch_results.xlsx'),index=True)
        filtered_results_df.to_excel(os.path.join(save_file_path, f'{savename}_GridSearch_Filtered_NoOverfit.xlsx'), index=False)
        pd.DataFrame({'loss': best_model.loss_curve_}).to_excel(os.path.join(save_file_path, f'{savename}_BestModel_LossCurve.xlsx'), index_label='Epoch')
        ##################################################################
        # Training MLP with best hyperparameter found from Grid Search.
        ##################################################################
        
        start_time_total_MLP = time.time()        
        optimized_model = MLPRegressor(
            hidden_layer_sizes=HIDDEN_LAYERS,
            learning_rate_init=LEARNING_RATE,
            batch_size=BATCH_SIZE,
            max_iter=NUM_EPOCHS,
            random_state=RANDOM_STATE
        )
        start_time_1_MLP=time.time()
        optimized_model.fit(X_train, y_train)
        end_time_1_MLP=time.time()
        elapsed_time_1_MLP= end_time_1_MLP - start_time_1_MLP
        
        

        start_time_2_MLP=time.time()
        y_train_pred_MLP = optimized_model.predict(X_train)
        end_time_2_MLP=time.time()
        elapsed_time_2_MLP= end_time_2_MLP - start_time_2_MLP
        
        start_time_3_MLP =time.time()
        y_test_pred_MLP = optimized_model.predict(X_test)
        end_time_3_MLP = time.time()
        elapsed_time_3_MLP = end_time_3_MLP - start_time_3_MLP
        
        y_actual_train_orig_MLP = scaler_y_NN.inverse_transform(y_train.reshape(1, -1))
        y_predicted_train_orig_MLP = scaler_y_NN.inverse_transform(y_train_pred_MLP.reshape(1, -1))
        y_actual_test_orig_MLP = scaler_y_NN.inverse_transform(y_test.reshape(1, -1))
        y_predicted_test_orig_MLP = scaler_y_NN.inverse_transform(y_test_pred_MLP.reshape(1, -1))

        n_train_MLP = y_actual_train_orig_MLP.size
        n_test_MLP = y_actual_test_orig_MLP.size
        
        total_layers_MLP = 1 + len(HIDDEN_LAYERS) + 1  # 1 input layer + hidden layers + 1 output layer.
        total_neurons_MLP = calculate_total_neurons(HIDDEN_LAYERS)
    
        num_iterations_MLP = (n_train_MLP) * NUM_EPOCHS
    
        # Calculating errors for the model.
         
        MAE_loss_train_MLP = mae(y_actual_train_orig_MLP, y_predicted_train_orig_MLP)
        MAE_loss_test_MLP = mae(y_actual_test_orig_MLP, y_predicted_test_orig_MLP)
        rmse_train_MLP = root_mean_squared_error(y_actual_train_orig_MLP, y_predicted_train_orig_MLP)
        rmse_test_MLP = root_mean_squared_error(y_actual_test_orig_MLP, y_predicted_test_orig_MLP)
        R2_score_train_MLP = R2(y_train, y_train_pred_MLP)
        R2_score_test_MLP = R2(y_test, y_test_pred_MLP)
        
        end_time_total_MLP = time.time()
        elapsed_time_total_MLP = end_time_total_MLP - start_time_total_MLP
        # Ploting actual vs train graphs for MLP and dynamically compute figure size.
        plt.figure(figsize=(8, 5))

        # Adjust sizes dynamically
        total_points = n_train_MLP + n_test_MLP
        min_scale = 0.5 
        max_scale = 2.5 
        base_size = 80
        scale_factor_MLP = np.clip(1000 / (total_points + 1), min_scale, max_scale)
        
        s_size = 80 * scale_factor_MLP
        
        plt.scatter(y_actual_train_orig_MLP.flatten(), y_predicted_train_orig_MLP.flatten(), color='red', edgecolor='black', s = s_size,
                marker ='o', label=fr'Training Data')
        plt.scatter(y_actual_test_orig_MLP.flatten(), y_predicted_test_orig_MLP.flatten(), marker ='^', s = s_size,
                    label=fr'Test Data', color='none', edgecolor='black')
        plt.plot([min(y), max(y)], [min(y), max(y)], color='black', alpha=0.8, linestyle='--', linewidth='1.3', label=fr'Actual Values')    
   
        
        plt.annotate(fr'R$^2$ Score (Train): {R2_score_train_MLP:.5f}', (0.015, 0.81), 
                     fontsize=11.5, xycoords='axes fraction')
        plt.annotate(fr'R$^2$ Score (Test): {R2_score_test_MLP:.5f}', (0.015, 0.74), 
                     fontsize=11.5, xycoords='axes fraction')
        plt.annotate(fr'MAE (Train): {MAE_loss_train_MLP:.2e} {Unit} (Test): {MAE_loss_test_MLP:.2e} {Unit}', 
             (0.015, 0.88), fontsize=11.5,  xycoords='axes fraction')
        plt.annotate(fr'RMSE (Train): {rmse_train_MLP:.2e} {Unit} (Test): {rmse_test_MLP:.2e} {Unit}', 
             (0.015, 0.95), fontsize=11.5,  xycoords='axes fraction')
        
        box_props = dict(boxstyle='round,pad=0.4', edgecolor='black', facecolor='white', alpha=0.1)
        textstr = '\n'.join((
            fr'MLP Hyperparameters',
            fr'',
            fr'Input Dimension: {num_params}',
            fr'Total Samples: {n_test_MLP + n_train_MLP}',
            fr'Train Samples: {n_train_MLP}',
            fr'Test Samples: {n_test_MLP}',
            fr'Total NN Layers: {total_layers_MLP}',
            fr'Total Neurons: {total_neurons_MLP}',
            fr'Learning Rate: {LEARNING_RATE}',
            fr'No. of Iterations: {NUM_EPOCHS}',
            fr'Random State: {RANDOM_STATE}',
            fr'Training time: {elapsed_time_1_MLP:.3f} s',
            fr'Inference time (Train): {elapsed_time_2_MLP:.3f} s',
            fr'Inference time (Test): {elapsed_time_3_MLP:.3f} s',
            fr'Total time taken: {elapsed_time_total_MLP:.3f} s'
            ))
        plt.annotate(textstr, (1.03, 0.20), fontsize=10, xycoords='axes fraction', bbox=box_props)
        
        
        plt.xlabel(fr'Actual Values ({y.name})', fontsize=16, fontweight='bold')
        plt.ylabel(f'Predicted Values ({y.name})', fontsize=16, fontweight='bold')
        plt.title(fr'{y.name} predictions from MLP', fontsize=14, fontweight='bold')
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.legend(loc='lower right', fontsize=12)
        plt.grid(False)       
        plt.tight_layout()
        
        # Saving the figure 
        plt.savefig(os.path.join(save_file_path, savename + '_MLP_Actual_vs_Prediceted_Data.jpg'), dpi=400)

        
        # Plot each metric over the course of the iterations (changing hyperparameters).
        metrics = ['mean_test_MAE', 'mean_test_RMSE', 'mean_test_R2']
        metrics_0 = ['mean_train_MAE','mean_train_RMSE', 'mean_train_R2']
        metrics_1 = ['MAE', 'RMSE', 'R2']
        metric_labels = ['MAE', 'Root Mean Squared Error (RMSE)', '$R^2$ Score']
        metric_labels_1 = ['MAE', 'RMSE', '$R^2$ Score'] 
        metric_labels_2 = ['MAE','RMS Error', 'R2 Score']
        colors = ['lime', 'blue', '#FF4500']
        
        markers = ['o', 'o', 'o'] 
        higher_is_better = [False, False, True]
        
        for metric, metric_0, metric_1, label, label_1, color, marker, is_higher_better, saving_name in zip(metrics, metrics_0, metrics_1, metric_labels, metric_labels_1, colors, markers, higher_is_better, metric_labels_2):
            
            optimum_score_index = results_df[f'rank_test_{metric_1}'].argmin()  # Index of the best score for each metric.
            optimum_params = results_df['params'][optimum_score_index]  # Best params corresponding to that score.
            optimum_score = results_df[f'mean_test_{metric_1}'][optimum_score_index]  # Best score for that metric.
            
            LEARNING_RATE_1 = optimum_params.get('learning_rate_init')
            NUM_EPOCHS_1 = optimum_params.get('max_iter')
            BATCH_SIZE_1 = optimum_params.get('batch_size')
            HIDDEN_LAYERS_1 = optimum_params.get('hidden_layer_sizes')

            # Dynamic plot size calculation
            base_width, base_height = 18, 6
            scale_w = 1 + 0.015 * (total_neurons_MLP // 10)
            scale_h = 1 + 0.015 * total_layers_MLP

            fig_width = min(14, base_width * scale_w)
            fig_height = min(10, base_height * scale_h)
            plt.figure(figsize=(fig_width, fig_height))

            # Calculate a scale factor relative to base size
            scale_factor = (fig_width * fig_height) / (base_width * base_height)

            # Adjust sizes dynamically
            marker_size = 10 * scale_factor
            marker_edge_width = 1.7 * scale_factor
            line_width_test = 1.5 * scale_factor
            line_width_train = 2 * scale_factor
            tick_fontsize = 22 * scale_factor
            label_fontsize = 26 * scale_factor
            legend_fontsize = 20 * scale_factor
            annot_fontsize = 14 * scale_factor
            hline_vline_width = 2.2 * scale_factor

            # Plotting
            plt.plot(results_df.index + 1, results_df[metric],
                     marker=marker, linestyle='-.', linewidth=line_width_test, alpha=0.35,
                     markersize=marker_size, markeredgecolor='black', markeredgewidth=marker_edge_width,
                     markerfacecolor=color, color='black', label='test', zorder=1)

            plt.plot(results_df.index + 1, results_df[metric_0],
                     marker='v', linestyle='-', linewidth=line_width_train, alpha=0.9,
                     markersize=marker_size * 1.1, markeredgecolor='black', markeredgewidth=marker_edge_width * 0.8,
                     markerfacecolor='white', color='black', label='train', zorder=0)

            # Best value and line indicators
            if is_higher_better:
                best_index = results_df[metric].idxmax()
                best_value = results_df[metric].max()
            else:
                best_index = results_df[metric].idxmin()
                best_value = results_df[metric].min()

            plt.axhline(best_value, color='red', linestyle='--', linewidth=hline_vline_width,
                        label=f'Best {label}: {best_value:.4f}', zorder=2)
            plt.axvline(best_index + 1, color='red', linestyle='--', linewidth=hline_vline_width, zorder=2)


            plt.xlabel("Iteration from Grid Search CV (Hyperparameter Combination)", fontsize=label_fontsize)
            plt.ylabel(label, fontsize=label_fontsize)
            plt.xticks(fontsize=tick_fontsize)
            plt.yticks(fontsize=tick_fontsize)

            # Legend
            if metric_1 in ['MAE', 'RMSE']:
                legend_loc = 'upper right'
            elif metric_1 in ['R2']:
                legend_loc = 'lower right'

            # Plot legend
            plt.legend(
                [f"{'Test'}", f"{'Train'}", f"Best {label_1}: {best_value:.4f}"],
                loc=legend_loc,
                fontsize=legend_fontsize,
                facecolor='white'
            )
            
            # Annotation box
            box_props = dict(
                boxstyle=f'round,pad={0.5 * scale_factor:.2f}',
                edgecolor='black',
                facecolor='white',
                alpha=0.25
            )
            if metric_1 in ['MAE', 'RMSE']:
                ann_xy = (0.02, 0.97)  
                ann_va = 'top'
            elif metric_1 in ['R2']:
                ann_xy = (0.02, 0.03)  
                ann_va = 'bottom'
            
            textstr = '\n'.join((
                fr'Best Hyperparameters ({label_1})',
                fr'',
                fr'Learning Rate: {LEARNING_RATE_1}',
                fr'Number of Iterations: {NUM_EPOCHS_1}',
                fr'Batch Size: {BATCH_SIZE_1}',
                fr'Hidden Layer: {HIDDEN_LAYERS_1}'
            ))

            plt.annotate(
                textstr,
                xy=ann_xy,  
                fontsize=annot_fontsize * 1.25,
                xycoords='axes fraction',
                bbox=box_props,
                verticalalignment=ann_va,
                horizontalalignment='left'
            )
            

            # Final layout and save
            plt.tight_layout()
            plt.savefig(os.path.join(save_file_path, f"{savename}_{saving_name.replace(' ', '_')}.jpg"), dpi=400)
        print("\n")

        # =====================================
        # Plot Best Model's 5-Fold CV Results
        # =====================================

        # Get the index of the best model according to refit metric (MAE in your case)
        best_idx = grid_search.best_index_
        
        # Extract per-fold test scores for R² and MAE
        # Note: MAE will be negative because greater_is_better=False, so take absolute value
        r2_scores = [
            cv_results[f'split{i}_test_R2'][best_idx]
            for i in range(5)
        ]
        mae_scores = [
            abs(cv_results[f'split{i}_test_MAE'][best_idx])
            for i in range(5)
        ]

        folds = np.arange(1, 6)  # Fold numbers 1–5
        width = 0.35

        fig, ax1 = plt.subplots(figsize=(5.5, 4.5))

        # Left axis: R²
        color_r2 = 'darkblue'
        ax1.bar(folds - width/2, r2_scores, width,
                color=color_r2, edgecolor='black', label=r'$R^2$ Score')
        ax1.set_xlabel('Fold Number', fontsize=14)
        ax1.set_ylabel(r'$R^2$ Score', color=color_r2, fontsize=14, labelpad=5)
        ax1.tick_params(axis='y', labelcolor=color_r2, labelsize=14)
        ax1.set_ylim([min(r2_scores)-0.02, 1.01])
        ax1.set_xticks(folds)
        ax1.set_xticklabels(folds, fontsize=14)
        ax1.locator_params(axis='y', nbins=4)

        # Right axis: MAE
        ax2 = ax1.twinx()
        color_mae = 'tab:red'
        ax2.bar(folds + width/2, mae_scores, width,
                color=color_mae, edgecolor='black', label='MAE')
        ax2.set_ylabel(f'MAE ({Unit})', color=color_mae, fontsize=14, labelpad=10)
        ax2.tick_params(axis='y', labelcolor=color_mae, labelsize=14)
        ax2.set_ylim([0, max(mae_scores)+0.03])
        ax2.locator_params(axis='y', nbins=5)

        # Combine legends from both axes
        lines_labels = [ax.get_legend_handles_labels() for ax in [ax1, ax2]]
        lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
        ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, 1.12),
                   ncol=2, fontsize=14, frameon=False)

        plt.tight_layout()
        plt.savefig(os.path.join(save_file_path, f"{savename}_Best5Fold_BarPlot_MLP.jpg"), dpi=700)
        plt.show()

        print("Mean R² (best model 5 folds): {:.4f} ± {:.4f}".format(np.mean(r2_scores), np.std(r2_scores)))
        print("Mean MAE (best model 5 folds): {:.4f} ± {:.4f}".format(np.mean(mae_scores), np.std(mae_scores)))
                      
        break

    else:
        print(f"{RED_TEXT}Invalid input. Please enter 'manual' or 'auto' to continue.{RESET_TEXT}")

print(f"{GREEN_TEXT} Grid Search completed.{RESET_TEXT}")
print("\n")

################################################################
# Deep Neural Network model building
################################################################

################################################################
# Function for DNN model building with optimized hyperparameters
################################################################

def build_model(input_shape):
    model = models.Sequential()

    # First layer
    model.add(layers.Dense(
        HIDDEN_LAYERS[0], 
        input_shape=(input_shape,),
        kernel_initializer='glorot_uniform',
        kernel_regularizer=regularizers.l2(0.0001)
    ))
    model.add(layers.PReLU())

    # Hidden layers
    for layer_size in HIDDEN_LAYERS[1:]:
        model.add(layers.Dense(
            layer_size,
            kernel_initializer='glorot_uniform',
            kernel_regularizer=regularizers.l2(0.0001)
        ))
        model.add(layers.PReLU())

    # Output layer for regression (no activation)
    model.add(layers.Dense(1))
    
    return model

################################################################
# Compile and train function for DNN
################################################################

def compile_and_train(model, X_train, y_train, X_test, y_test, 
                      learning_rate=LEARNING_RATE, num_epochs=NUM_EPOCHS, 
                      batch_size=BATCH_SIZE, max_wall_time_hours=2):
    """
    Complie and train function with multiple termination conditions:
    1. Maximum epoch limit (1500)
    2. Validation loss threshold (1e-6) with patience window (5 epochs)
    3. Maximum wall-clock time (2 hours)
    4. Early stopping based on validation loss improvement
    """
    
    optimizer = optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-5)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Multiple termination callbacks
    callbacks_list = []
    
    # 1. Early stopping for validation loss improvement
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=1e-4,
        patience=40,
        restore_best_weights=True,
        verbose=1
    )
    callbacks_list.append(early_stop)
    
    # 2. Custom callback for validation loss threshold with patience
    class ValidationLossThresholdCallback(callbacks.Callback):
        def __init__(self, threshold=1e-6, patience=50):
            super().__init__()
            self.threshold = threshold
            self.patience = patience
            self.wait = 0
            
        def on_epoch_end(self, epoch, logs=None):
            current_val_loss = logs.get('val_loss')
            if current_val_loss is not None:
                if current_val_loss < self.threshold:
                    self.wait += 1
                    print(f"\nValidation loss {current_val_loss:.2e} below threshold {self.threshold:.2e}. "
                          f"Patience: {self.wait}/{self.patience}")
                    if self.wait >= self.patience:
                        print(f"\nTerminating: Validation loss below {self.threshold:.2e} for {self.patience} epochs")
                        self.model.stop_training = True
                else:
                    self.wait = 0
    
    val_threshold_callback = ValidationLossThresholdCallback(threshold=1e-3, patience=100)
    callbacks_list.append(val_threshold_callback)
    
    # 3. Custom callback for maximum wall-clock time
    class WallTimeCallback(callbacks.Callback):
        def __init__(self, max_time_hours=2):
            super().__init__()
            self.max_time_seconds = max_time_hours * 3600
            self.start_time = None
            
        def on_train_begin(self, logs=None):
            self.start_time = time.time()
            print(f"Training started. Maximum wall-clock time: {self.max_time_seconds/3600:.1f} hours")
            
        def on_epoch_end(self, epoch, logs=None):
            if self.start_time is not None:
                elapsed_time = time.time() - self.start_time
                if elapsed_time > self.max_time_seconds:
                    print(f"\nTerminating: Maximum wall-clock time ({self.max_time_seconds/3600:.1f} hours) exceeded")
                    print(f"Elapsed time: {elapsed_time/3600:.2f} hours")
                    self.model.stop_training = True
    
    wall_time_callback = WallTimeCallback(max_time_hours=max_wall_time_hours)
    callbacks_list.append(wall_time_callback)
    
    # 4. Custom callback for maximum epoch limit
    class MaxEpochCallback(callbacks.Callback):
        def __init__(self, max_epochs=1500):
            super().__init__()
            self.max_epochs = max_epochs
            
        def on_epoch_end(self, epoch, logs=None):
            if epoch + 1 >= self.max_epochs:
                print(f"\nTerminating: Maximum epoch limit ({self.max_epochs}) reached")
                self.model.stop_training = True
    
    max_epoch_callback = MaxEpochCallback(max_epochs=1500)
    callbacks_list.append(max_epoch_callback)
    
    # Training with all termination conditions
    start_time = time.time()
    print("\n" + "="*60)
    print("TRAINING WITH TERMINATION CONDITIONS")
    print("="*60)
    print("Termination conditions:")
    print("1. Maximum epochs: 1500")
    print("2. Validation loss < 1e-6 for 5 consecutive epochs")
    print("3. Maximum wall-clock time: 2 hours")
    print("4. Early stopping: validation loss improvement (patience=10)")
    print("="*60)
    
    try:
        history = model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=min(num_epochs, 1500),  # Respect the maximum epoch limit
            batch_size=batch_size,
            verbose=1,
            callbacks=callbacks_list
        )
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        
        # Report termination reason
        print("\n" + "="*60)
        print("TRAINING COMPLETED")
        print("="*60)
        print(f"Training time: {elapsed_time:.2f} seconds ({elapsed_time/3600:.2f} hours)")
        print(f"Total epochs completed: {len(history.history['loss'])}")
        
        # Determine termination reason
        total_epochs = len(history.history['loss'])
        final_val_loss = history.history['val_loss'][-1] if 'val_loss' in history.history else None
        
        if total_epochs >= 1500:
            print("Termination reason: Maximum epoch limit (1500) reached")
        elif elapsed_time > max_wall_time_hours * 3600:
            print(f"Termination reason: Maximum wall-clock time ({max_wall_time_hours} hours) exceeded")
        elif final_val_loss is not None and final_val_loss < 1e-6:
            print("Termination reason: Validation loss threshold (1e-6) reached with patience")
        else:
            print("Termination reason: Early stopping due to validation loss plateau")
        
        print(f"Final validation loss: {final_val_loss:.2e}" if final_val_loss else "No validation loss recorded")
        print("="*60)
        
        return history, elapsed_time
        
    except Exception as e:
        print(f"Training interrupted due to error: {str(e)}")
        end_time = time.time()
        elapsed_time = end_time - start_time
        return None, elapsed_time


################################################################
# Function for calculating error in the prediction from DNN
################################################################

def evaluate_model(model, X_test, y_test, X_train, y_train):
    start_time = time.time()
    predicted_train = model.predict(X_train, verbose=0)
    end_time = time.time()
    elapsed_time_2 = end_time - start_time
    print(f"Inference time (Train): {elapsed_time_2:.2f} seconds")
    
    start_time = time.time()
    predicted_test = model.predict(X_test, verbose=0)
    end_time = time.time()
    elapsed_time_3 = end_time - start_time
    print(f"Inference time (Test): {elapsed_time_3:.2f} seconds")
    
    y_actual_train_orig = scaler_y_NN.inverse_transform(y_train.reshape(1, -1))
    y_predicted_train_orig = scaler_y_NN.inverse_transform(predicted_train.reshape(1, -1))
    y_actual_test_orig = scaler_y_NN.inverse_transform(y_test.reshape(1, -1))
    y_predicted_test_orig = scaler_y_NN.inverse_transform(predicted_test.reshape(1, -1))

    
    MAE_loss_train_DNN = mae(y_actual_train_orig, y_predicted_train_orig)
    MAE_loss_test_DNN = mae(y_actual_test_orig, y_predicted_test_orig)
    rmse_train_DNN = root_mean_squared_error(y_actual_train_orig, y_predicted_train_orig)
    rmse_test_DNN = root_mean_squared_error(y_actual_test_orig, y_predicted_test_orig)
    R2_score_train_DNN = R2(y_train, predicted_train)
    R2_score_test_DNN = R2(y_test, predicted_test)
    
    return elapsed_time_2, elapsed_time_3, MAE_loss_train_DNN, MAE_loss_test_DNN, rmse_train_DNN, rmse_test_DNN, R2_score_train_DNN, R2_score_test_DNN, predicted_train, predicted_test

###################################################################################
# Function to Save the DNN model for further use (like during optimization)
###################################################################################

def save_model(model, scaler_X_NN, scaler_y_NN, save_file_path, model_name):
    save_folder = os.path.join(save_file_path, savename)
    os.makedirs(save_folder, exist_ok=True) 
    
    model_path = os.path.join(save_folder, f"{savename}_{model_name}.keras")
    model.save(model_path)

###################################################################################
# Function to obtain prediction from DNN in an excel file with original scale
###################################################################################

def save_results_to_excel(y_actual_train, predicted_train, y_actual_test, predicted_test, MAE_loss_train_DNN, MAE_loss_test_DNN, rmse_train_DNN, rmse_test_DNN, R2_score_train_DNN, R2_score_test_DNN, target_name, save_file_path, savename, scaler_X_NN, scaler_y_NN):
    
    y_actual_train_orig = scaler_y_NN.inverse_transform(y_actual_train)
    predicted_train_orig = scaler_y_NN.inverse_transform(predicted_train)

    results_train_dict = {
        'Actual Train Values from DNN': y_actual_train_orig.flatten(),
        'Predicted Train Values from DNN': predicted_train_orig.flatten(),
    }
    results_train_df = pd.DataFrame(results_train_dict)
    results_train_df.to_excel(os.path.join(save_file_path, f'{savename}_DNN_prediceted_train_data_{target_name}.xlsx'),index=False)

    y_actual_test_orig = scaler_y_NN.inverse_transform(y_actual_test)
    predicted_test_orig = scaler_y_NN.inverse_transform(predicted_test)

    results_test_dict = {
        'Actual Test Values from DNN': y_actual_test_orig.flatten(),
        'Predicted Test Values from DNN': predicted_test_orig.flatten()
    }
    results_test_df = pd.DataFrame(results_test_dict)
    results_test_df.to_excel(os.path.join(save_file_path, f'{savename}_DNN_prediceted_test_data_{target_name}.xlsx'),index=False)

################################################################
# Function for plotting Actual vs Prediction Graph for DNN
################################################################

def plot_results(elapsed_time_1, elapsed_time_2, elapsed_time_3, elapsed_time_total_DNN, 
                 X, y_actual_train, y_predicted_train, y_actual_test, y_predicted_test, 
                 MAE_loss_train_DNN, MAE_loss_test_DNN, rmse_train_DNN, rmse_test_DNN, 
                 R2_score_train_DNN, R2_score_test_DNN, 
                 scaler_X_NN, scaler_y_NN, target_name, savename):

    # Inverse transform
    y_actual_train_orig = scaler_y_NN.inverse_transform(y_actual_train)
    y_predicted_train_orig = scaler_y_NN.inverse_transform(y_predicted_train)
    y_actual_test_orig = scaler_y_NN.inverse_transform(y_actual_test)
    y_predicted_test_orig = scaler_y_NN.inverse_transform(y_predicted_test)

    # Dynamic plot sizing based on sample size
    n_train = y_actual_train.size
    n_test = y_actual_test.size
    total_layers = 1 + len(HIDDEN_LAYERS) + 1
    total_neurons = calculate_total_neurons(HIDDEN_LAYERS)
    num_iterations = n_train * NUM_EPOCHS

    # Dynamic figsize logic
   
    plt.figure(figsize=(8,5))

    # Adjust sizes dynamically
    total_layers_DNN = n_train + n_test
    min_scale = 0.5 
    max_scale = 2.5 
    base_size = 80
    scale_factor_DNN = np.clip(1000 / (total_layers_DNN + 1), min_scale, max_scale)
        
    # scale_factor_DNN = 1 / np.log10(total_layers_DNN + 10)
    s_size_DNN = 80 * scale_factor_DNN
    # Error metrics and annotations
    plt.annotate(fr'R$^2$ Score (Train): {R2_score_train_DNN:.5f}', (0.015, 0.81), fontsize=11.5, xycoords='axes fraction')
    plt.annotate(fr'R$^2$ Score (Test): {R2_score_test_DNN:.5f}', (0.015, 0.74), fontsize=11.5, xycoords='axes fraction')
    plt.annotate(fr'MAE (Train): {MAE_loss_train_DNN:.2e} {Unit} (Test): {MAE_loss_test_DNN:.2e} {Unit}', (0.015, 0.88), fontsize=11.5, xycoords='axes fraction')
    plt.annotate(fr'RMSE (Train): {rmse_train_DNN:.2e} {Unit} (Test): {rmse_test_DNN:.2e} {Unit}', (0.015, 0.95), fontsize=11.5, xycoords='axes fraction')

    # Scatter plots
    plt.scatter(y_actual_train_orig.flatten(), y_predicted_train_orig.flatten(), color='royalblue', edgecolor='black', s = s_size_DNN,
                marker='o', label='Training Data')
    plt.scatter(y_actual_test_orig.flatten(), y_predicted_test_orig.flatten(), marker='^', s = s_size_DNN,
                label='Test Data', color='none', edgecolor='black')
    plt.plot([min(y), max(y)], [min(y), max(y)], color='black', alpha=0.8, linestyle='--', linewidth=1.7, label='Actual Values')

    # Box with model parameters
    box_props = dict(boxstyle='round,pad=0.4', edgecolor='black', facecolor='white', alpha=0.1)
    textstr = '\n'.join((
        'DNN Hyperparameters',
        '',
        fr'Input Dimension: {num_params}',
        fr'Total Samples: {n_test + n_train}',
        fr'Train Samples: {n_train}',
        fr'Test Samples: {n_test}',
        fr'Total NN Layers: {total_layers}',
        fr'Total Neurons: {total_neurons}',
        fr'Learning Rate: {LEARNING_RATE}',
        fr'No. of Iterations: {NUM_EPOCHS}',
        fr'Random State: {RANDOM_STATE}',
        fr'Training time: {elapsed_time_1:.3f} s',
        fr'Inference time (Train): {elapsed_time_2:.3f} s',
        fr'Inference time (Test): {elapsed_time_3:.3f} s',
        fr'Total time taken: {elapsed_time_total_DNN:.3f} s'
    ))
    plt.annotate(textstr, (1.03, 0.20), fontsize=10, xycoords='axes fraction', bbox=box_props)

    plt.xlabel(f'Actual Values ({y.name})', fontsize=16, fontweight='bold')
    plt.ylabel(f'Predicted Values ({y.name})', fontsize=16, fontweight='bold')
    plt.title(f'{y.name} predictions from DNN with PReLU activation', fontsize=14, fontweight='bold')
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(loc='lower right', fontsize=12)
    plt.grid(False)
    plt.tight_layout()

    # Save the figure
    plt.savefig(os.path.join(save_file_path, savename + '_DNN_Actual_vs_Prediceted_Data.jpg'), dpi=400)

################################################################
# Function to plot the errors with iterations with Adam in DNN
################################################################
def plot_loss(history, savename, termination_info=None):
       
    if history is None:
        print("No training history available for plotting.")
        return
        
    loss_data = history.history['loss']
    val_loss_data = history.history.get('val_loss', [])
    
    final_train_loss = loss_data[-1]
    final_val_loss = val_loss_data[-1] if val_loss_data else None
    total_epochs = len(loss_data)

    indices = range(0, len(loss_data), 3)
    points = max(1, int(len(loss_data) * 0.05))

    indices_train = list(range(0, len(loss_data), points))
    indices_val = list(range(0, len(val_loss_data), points)) if val_loss_data else []

    plt.figure(figsize=(15, 8))

    if val_loss_data:
        plt.plot(
            range(len(val_loss_data)),
            val_loss_data,
            linestyle='-',
            color='darkgrey',
            label=f'Validation Loss (Final: {final_val_loss:.3e})'
        )
        plt.plot(
            indices_val,
            [val_loss_data[i] for i in indices_val],
            linestyle='',
            color='royalblue',
            marker='^',
            markersize=10,
            markerfacecolor='white',
            markeredgecolor='grey',
            markeredgewidth=2
        )

    plt.plot(
        indices,
        [loss_data[i] for i in indices],
        linestyle='-',
        color='royalblue',
        linewidth=1.5,
        marker='',
        markersize=12,
        markeredgecolor='purple',
        label=f'Training Loss (Final: {final_train_loss:.3e})'
    )

    plt.plot(
        indices_train,
        [loss_data[i] for i in indices_train],
        linestyle='',
        color='royalblue',
        marker='o',
        markersize=10,
        markerfacecolor='white',
        markeredgecolor='purple',
        markeredgewidth=2
    )

    # Add termination condition information
    title = fr'DNN (PReLU) - Loss vs. Epoch for ({y.name})'
    subtitle = f'Completed {total_epochs} epochs'
    
    # Add horizontal line for validation loss threshold
    if val_loss_data:
        plt.axhline(y=1e-6, color='red', linestyle='--', alpha=0.7, 
                   label='Val Loss Threshold (1e-6)')

    plt.title(title, fontsize=24, fontweight='bold')
    plt.text(0.5, 0.95, subtitle, transform=plt.gca().transAxes, 
             fontsize=14, ha='center', va='top')
    
    plt.xlabel('Epoch', fontsize=20, fontweight='bold')
    plt.ylabel('Loss', fontsize=20, fontweight='bold')
    plt.legend(fontsize=16)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.tight_layout()

    # Save the figure
    plt.savefig(os.path.join(save_file_path, savename + '_DNN_loss_vs_epoch.jpg'), 
                dpi=400, bbox_inches='tight')
    plt.show()

################################################################
# Function to perform 5-fold CV with the DNN predicted results
################################################################
def plot_5fold_cv(model, X, y):

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2_scores = []
    mae_scores = []
    
    fold = 1
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        r2_scores.append(r2_score(y_test, y_pred))
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        fold += 1
    
    # Plot
    folds = np.arange(1, 6)
    width = 0.35

    fig, ax1 = plt.subplots(figsize=(5.5, 4.5))

    # Left axis - R²
    color_r2 = 'darkblue'
    ax1.bar(folds - width/2, r2_scores, width, color=color_r2, edgecolor='black', label='R² Score')
    ax1.set_xlabel('Fold Number', fontsize=14)
    ax1.set_ylabel('R² Score', fontsize=14, color=color_r2)
    ax1.tick_params(axis='y', labelcolor=color_r2)

    # Right axis - MAE
    ax2 = ax1.twinx()
    color_mae = 'darkred'
    ax2.bar(folds + width/2, mae_scores, width, color=color_mae, edgecolor='black', label='MAE')
    ax2.set_ylabel('Mean Absolute Error', fontsize=14, color=color_mae)
    ax2.tick_params(axis='y', labelcolor=color_mae)

    # Legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

    plt.title('5-Fold CV Performance', fontsize=16)
    plt.tight_layout()
    plt.show()

################################################################
# Function to Save data for later optimization
################################################################

def save_model_with_metadata(model, scaler_X, scaler_y, save_file_path, savename, model_name, 
                             input_features, output_features, X_data, y_data):
    save_folder = os.path.join(save_file_path, savename)
    os.makedirs(save_folder, exist_ok=True)

    # Compute min/max directly from data
    X_min = X_data.min(axis=0).tolist()
    X_max = X_data.max(axis=0).tolist()
    y_min = y_data.min(axis=0).tolist() if y_data is not None else None
    y_max = y_data.max(axis=0).tolist() if y_data is not None else None

    # Save metadata
    metadata = {
        "model_name": model_name,
        "scaler_X_path": scaler_X_path,
        "scaler_y_path": scaler_y_path,
        "input_features": list(input_features),
        "output_features": list(output_features),
        "scaler_X_mean": scaler_X.mean_.tolist(),
        "scaler_X_scale": scaler_X.scale_.tolist(),
        "scaler_y_mean": scaler_y.mean_.tolist() if hasattr(scaler_y, "mean_") else None,
        "scaler_y_scale": scaler_y.scale_.tolist() if hasattr(scaler_y, "scale_") else None,
        "scaler_X_min": X_min,
        "scaler_X_max": X_max,
        "scaler_y_min": y_min,
        "scaler_y_max": y_max,
    }

    metadata_path = os.path.join(save_folder, f"{savename}_metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"Model, scalers, and metadata saved at: {save_folder}")




def save_metadata_to_excel(save_file_path, savename, metadata_list):
    
    excel_path = os.path.join(save_file_path, savename, f"{savename}_model_info.xlsx")
    
    # Flatten metadata for Excel
    rows = []
    for md in metadata_list:
        rows.append({
            "Model Name": md["model_name"],
            "Model Path": md["model_path"],
            "Scaler X Path": md["scaler_X_path"],
            "Scaler Y Path": md["scaler_y_path"],
            "Input Features": ", ".join(md["input_features"]),
            "Output Features": ", ".join(md["output_features"]),
            "Scaler X Min": md["scaler_X_min"],
            "Scaler X Max": md["scaler_X_max"],
            "Scaler Y Min": md["scaler_y_min"],
            "Scaler Y Max": md["scaler_y_max"],
        })
    
    df = pd.DataFrame(rows)
    df.to_excel(excel_path, index=False)
    print(f"Metadata saved to Excel: {excel_path}")

###################
# Model execution
###################

print(f"{BLUE_TEXT} Starting DNN...{RESET_TEXT}")
print("\n")

start_time_total_DNN = time.time() 

# Building the deep neural network model
model = build_model(X_train.shape[1])

# Training with termination conditions
history, elapsed_time_1 = compile_and_train(
    model, X_train, y_train, X_test, y_test,
    learning_rate=LEARNING_RATE,
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    max_wall_time_hours=2  # 2 hours maximum
)

# Continue with evaluation only if training was successful
if history is not None:
    # Calculate errors in the predictions from DNN
    elapsed_time_2, elapsed_time_3, MAE_loss_train_DNN, MAE_loss_test_DNN, \
    rmse_train_DNN, rmse_test_DNN, R2_score_train_DNN, R2_score_test_DNN, \
    predicted_train, predicted_test = evaluate_model(model, X_test, y_test, X_train, y_train)

    end_time_total_DNN = time.time()
    elapsed_time_total_DNN = end_time_total_DNN - start_time_total_DNN

    # Save the trained DNN model
    save_model(model, scaler_X_NN, scaler_y_NN, save_file_path, 'trained_model')

    # Save the scalers
    joblib.dump(scaler_X_NN, os.path.join(save_file_path, savename, savename + '_scaler_X.joblib'))
    joblib.dump(scaler_y_NN, os.path.join(save_file_path, savename, savename + '_scaler_y.joblib'))

    # Save predictions from DNN
    save_results_to_excel(y_train, predicted_train, y_test, predicted_test, 
                         MAE_loss_train_DNN, MAE_loss_test_DNN, rmse_train_DNN, rmse_test_DNN, 
                         R2_score_train_DNN, R2_score_test_DNN, 'Input_vs_Prediction', 
                         save_file_path, savename, scaler_X_NN, scaler_y_NN)

    # Plot results
    plot_results(elapsed_time_1, elapsed_time_2, elapsed_time_3, elapsed_time_total_DNN, 
                X_scaled, y_train, predicted_train, y_test, predicted_test, 
                MAE_loss_train_DNN, MAE_loss_test_DNN, rmse_train_DNN, rmse_test_DNN, 
                R2_score_train_DNN, R2_score_test_DNN, scaler_X_NN, scaler_y_NN, 
                'Input_vs_Prediction', savename)

    # Plot loss curve
    plot_loss(history, savename)

    # Perform 5-fold CV
    plot_5fold_cv(model, X_scaled, y_scaled)

    # Saving model in json format
    metadata_list = []

    # Save model + scalers + metadata
    save_model_with_metadata(
        model, 
        scaler_X_NN, 
        scaler_y_NN, 
        save_file_path,
        savename,
        model_name="DeepModel",
        input_features=X.columns, 
        output_features=[y.name],
        X_data=X.values, 
        y_data=y.values
        )    


    # Load the saved metadata json for Excel export
    with open(os.path.join(save_file_path, savename, f"{savename}_metadata.json")) as f:
        metadata = json.load(f)
    metadata_list.append(metadata)

    # Save all metadata to Excel
    save_metadata_to_excel(save_file_path, savename, metadata_list)

    print(f"{GREEN_TEXT}DNN training completed successfully!{RESET_TEXT}")
    
else:
    print(f"{RED_TEXT}Training failed. Please check the error messages above.{RESET_TEXT}")

end_time_total = time.time()
elapsed_time_total = end_time_total - start_time_total

print(f"{BLUE_TEXT}\nTotal runtime: {elapsed_time_total:.2f} seconds{RESET_TEXT}")
print(f"{GREEN_TEXT}Completed{RESET_TEXT}")